In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings

warnings .filterwarnings('ignore')
%matplotlib inline

In [3]:
df = pd.read_csv('cardekho_imputated.csv',index_col=[0])

In [4]:
df.head()

,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [5]:
df.drop(columns=['car_name','brand'],inplace=True)

In [6]:
df.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [7]:
len(df['model'].unique())

120

In [8]:
cat_features = df.select_dtypes(include='object').columns
num_feature = df.select_dtypes(exclude='object').columns
discrete_fea = [ feature for feature in num_feature if len(df[feature].unique())<=25]
continueous_fea = [ feature for feature in num_feature if len(df[feature].unique())>25]

In [9]:
print('object feature :' , len(cat_features))
print('numerical feature :' , len(num_feature))
print('discrete feature :' , len(discrete_fea))
print('contineous feature :' , len(continueous_fea))

object feature : 4
numerical feature : 7
discrete feature : 2
contineous feature : 5


In [10]:
from sklearn.model_selection import train_test_split
x = df.drop('selling_price',axis=1)
y= df['selling_price']

In [11]:
# doning label encoding for the model column as it has many unique elements
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x['model']=le.fit_transform(x['model'])

In [12]:
x.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,7,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,54,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,118,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,7,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,38,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


In [13]:
(len(df['seller_type'].unique()), len(df['transmission_type'].unique()), len(df['fuel_type'].unique()))

(3, 2, 5)

##### here we can do one hot encoding as there are limited no of features

In [14]:
cat_features = ['seller_type','transmission_type','fuel_type']
num_feature = x.select_dtypes(exclude='object').columns

In [15]:
num_feature


Index(['model', 'vehicle_age', 'km_driven', 'mileage', 'engine', 'max_power',
       'seats'],
      dtype='object')

In [16]:
cat_features

['seller_type', 'transmission_type', 'fuel_type']

In [17]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

scaler  = StandardScaler()
encoder = OneHotEncoder(drop='first')

transformer = ColumnTransformer(
    [
       ("OneHotEncoder",encoder,cat_features),
       ("StandardScaler", scaler,num_feature)
    ],remainder="passthrough"
)

x = transformer.fit_transform(x)

In [18]:
x

array([[ 1.        ,  0.        ,  1.        , ..., -1.32425883,
        -1.26335238, -0.40302241],
       [ 1.        ,  0.        ,  1.        , ..., -0.55471774,
        -0.43257082, -0.40302241],
       [ 1.        ,  0.        ,  1.        , ..., -0.55471774,
        -0.47911321, -0.40302241],
       ...,
       [ 0.        ,  0.        ,  1.        , ...,  0.02291783,
         0.06822523, -0.40302241],
       [ 0.        ,  0.        ,  1.        , ...,  1.32979434,
         0.91715831,  2.07344426],
       [ 0.        ,  0.        ,  0.        , ...,  0.02099878,
         0.39588361, -0.40302241]])

In [19]:
pd.DataFrame(x)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,-1.519714,0.983562,1.247335,-0.000276,-1.324259,-1.263352,-0.403022
1,1.0,0.0,1.0,0.0,0.0,0.0,1.0,-0.225693,-0.343933,-0.690016,-0.192071,-0.554718,-0.432571,-0.403022
2,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.536377,1.647309,0.084924,-0.647583,-0.554718,-0.479113,-0.403022
3,1.0,0.0,1.0,0.0,0.0,0.0,1.0,-1.519714,0.983562,-0.360667,0.292211,-0.936610,-0.779312,-0.403022
4,0.0,0.0,1.0,1.0,0.0,0.0,0.0,-0.666211,-0.012060,-0.496281,0.735736,0.022918,-0.046502,-0.403022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15406,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.508844,0.983562,-0.869744,0.026096,-0.767733,-0.757204,-0.403022
15407,0.0,0.0,1.0,0.0,0.0,0.0,1.0,-0.556082,-1.339555,-0.728763,-0.527711,-0.216964,-0.220803,2.073444
15408,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.407551,-0.012060,0.220539,0.344954,0.022918,0.068225,-0.403022
15409,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.426247,-0.343933,72.541850,-0.887326,1.329794,0.917158,2.073444


In [20]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3)

## model training and model selection

In [21]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [22]:
def model_evaluate(test,predicted):
    mae = mean_absolute_error(test,predicted)
    mse = mean_squared_error(test,predicted)
    rmse = np.sqrt(mse)
    r2score = r2_score(test,predicted)
    return mae,rmse,r2score

In [23]:
models = {
    "liniear Reagression":LinearRegression(),
    "Lasso":Lasso(),
    "Ridge":Ridge(),
    "RandomForestRegressor":RandomForestRegressor(),
    "Decision Tree Regressor":DecisionTreeRegressor(),
    "KNeighborsRegressor":KNeighborsRegressor(),
    "Gradient Boost Regressor":GradientBoostingRegressor()
}

In [24]:
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(x_train,y_train)

    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    train_mae,train_rmse,train_r2score = model_evaluate(y_train,y_train_pred)
    test_mae,test_rmse,test_r2score = model_evaluate(y_test,y_test_pred)

    print(list(models.keys())[i])
    print('Model Performence for the traing data')
    print('- root_mean_squared_error:', train_rmse)
    print('- mean_absolute_error:', train_mae)
    print('-r2_score:', train_r2score)
    print('-'*35)
    
    print('Model Performence for the test data')
    print('- root_mean_squared_error:', test_rmse)
    print('- mean_absolute_error:', test_mae)
    print('-r2_score:', test_r2score)
    print('='*35)
    print('\n')


liniear Reagression
Model Performence for the traing data
- root_mean_squared_error: 566211.5303353169
- mean_absolute_error: 267691.55900889204
-r2_score: 0.6144663108757713
-----------------------------------
Model Performence for the test data
- root_mean_squared_error: 488902.3280236588
- mean_absolute_error: 271083.03425842855
-r2_score: 0.6700021972176483


Lasso
Model Performence for the traing data
- root_mean_squared_error: 566211.5357914943
- mean_absolute_error: 267689.888038772
-r2_score: 0.6144663034455439
-----------------------------------
Model Performence for the test data
- root_mean_squared_error: 488902.4653235306
- mean_absolute_error: 271083.71329454845
-r2_score: 0.6700020118691246


Ridge
Model Performence for the traing data
- root_mean_squared_error: 566212.0463177006
- mean_absolute_error: 267656.3289609929
-r2_score: 0.6144656082101705
-----------------------------------
Model Performence for the test data
- root_mean_squared_error: 488900.6476292884
- mean_

#### for the models which have high accuracy we will perform hyperparameter tunning

In [27]:

rf_param = {
   'max_depth':[1,2,3,4,5,8,15,None,10],
   'max_features':['auto','sqrt','log2'],
   'n_estimators':[100,200,500],
   'min_samples_split':[2,8,15,20]
}
GradientBoostingRegressor()
gb_param= {
    "loss":['squared_error', 'absolute_error', 'huber', 'quantile'],
    "n_estimators": [100,200,500,700],
    "criterion":['friedman_mse', 'squared_error','mse'] ,
    "min_samples_split": [2,8,15,20],
    "max_depth":[5,8,15,None,10],
    "learning_rate":[0.1,0.01,0.02,0.03,0.04]
}

In [28]:
## tunning for knn ans random forest

tunnedModel = [
    
    ("RandomForestRegressor",RandomForestRegressor(),rf_param),
    ("gradientBoostRegressor",GradientBoostingRegressor(),gb_param)
]

from sklearn.model_selection import RandomizedSearchCV
model_params={}
for name , model ,param in tunnedModel:
    random = RandomizedSearchCV(estimator=model,param_distributions=param,cv=3,verbose=3,n_jobs=-1)
    random.fit(x_train,y_train)
    model_params[name]=random.best_params_
for model_name in model_params:
    print(f'---------------Best Params for {model_name}-------------------')
    print(model_params[model_name])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits
---------------Best Params for RandomForestRegressor-------------------
{'n_estimators': 500, 'min_samples_split': 8, 'max_features': 'sqrt', 'max_depth': None}
---------------Best Params for gradientBoostRegressor-------------------
{'n_estimators': 500, 'min_samples_split': 2, 'max_depth': 15, 'loss': 'absolute_error', 'learning_rate': 0.04, 'criterion': 'squared_error'}


In [29]:
tunned_models={
    "randomForsest":RandomForestRegressor(n_estimators=500,min_samples_split=2, max_features= 'sqrt', max_depth= None),
    "GradientBoostingRegressor":GradientBoostingRegressor(n_estimators=500, min_samples_split= 2, max_depth= 15, loss='absolute_error', learning_rate=0.04, criterion= 'squared_error')
  
}

In [30]:
for i in range(len(list(tunned_models))):
    model = list(tunned_models.values())[i]
    model.fit(x_train,y_train)

    y_train_pred1 = model.predict(x_train)
    y_test_pred1 = model.predict(x_test)

    train_mae1,train_rmse1,train_r2score1 = model_evaluate(y_train,y_train_pred1)
    test_mae1,test_rmse1,test_r2score1 = model_evaluate(y_test,y_test_pred1)

    print(list(tunned_models.keys())[i])
    print('Model Performence for the traing data')
    print('- root_mean_squared_error:', train_rmse1)
    print('- mean_absolute_error:', train_mae1)
    print('-r2_score:', train_r2score1)
    print('-'*35)
    
    print('Model Performence for the test data')
    print('- root_mean_squared_error:', test_rmse1)
    print('- mean_absolute_error:', test_mae1)
    print('-r2_score:', test_r2score1)
    print('='*35)
    print('\n')


randomForsest
Model Performence for the traing data
- root_mean_squared_error: 142946.20823056728
- mean_absolute_error: 39511.088205705295
-r2_score: 0.9754274518080959
-----------------------------------
Model Performence for the test data
- root_mean_squared_error: 230919.66607322867
- mean_absolute_error: 100677.33366374954
-r2_score: 0.9263812092847148


GradientBoostingRegressor
Model Performence for the traing data
- root_mean_squared_error: 80137.65110931137
- mean_absolute_error: 33116.04201535653
-r2_score: 0.9922771405077945
-----------------------------------
Model Performence for the test data
- root_mean_squared_error: 228612.4029693807
- mean_absolute_error: 100474.62300898126
-r2_score: 0.9278450026807308


